In [1]:
17

17

In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import tokenization

In [3]:
COMMENT = 'comment_text'
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [4]:
base_path = 'jigsaw-toxic-comment-classification-challenge'
train = pd.read_csv(f'{base_path}/train.csv')
test = pd.read_csv(f'{base_path}/test.csv')
submission = pd.read_csv(f'{base_path}/sample_submission.csv')
test_labels = pd.read_csv(f'{base_path}/test_labels.csv')

In [5]:
EMBEDDING_FILE = f'{base_path}/glove.6B.300d.txt'
def get_coefs(word,*arr): 
    return word.lower(), np.asarray(arr, dtype='float32')

with open(EMBEDDING_FILE) as f:
    embeddings_index_1 = dict(get_coefs(*o.strip().split()) for o in f)

all_embs_1 = np.stack(embeddings_index_1.values())
emb_mean_1 = all_embs_1.mean()
emb_std_1 = all_embs_1.std()
emb_mean_1, emb_std_1

/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(-0.0039050116, 0.38177028)

In [6]:
EMBEDDING_FILE = f'{base_path}/GoogleNews-vectors-negative300.bin'

wv = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
embeddings_index_2 = {w.lower():wv.get_vector(w) for w in wv.index2word}

all_embs_2 = np.stack(embeddings_index_2.values())
emb_mean_2 = all_embs_2.mean()
emb_std_2 = all_embs_2.std()
emb_mean_2, emb_std_2

/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(-0.0033393223, 0.1248688)

In [7]:
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')
embeddings_index_3 = {w.lower():ft.get_word_vector(w) for w in ft.get_words()}
all_embs_3 = np.stack(embeddings_index_3.values())
emb_mean_3 = all_embs_3.mean()
emb_std_3 = all_embs_3.std()
emb_mean_3, emb_std_3

/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.0020054176, 0.06903591)

# Problem understanding

# Dataset generation

# Metrics define

# Validation strategy

# Data processing (extract useful information)

In [8]:
import sys
sys.path.append('/home/shanger_lin/python3.6/ShangerUtility/')
from spellcheck import Corrector

word_freq = '/home/shanger_lin/python3.6/ShangerUtility/frequency_dictionary_en_82_765.txt'
C = Corrector(word_freq)

In [9]:
import re
import textwrap
import unicodedata
import warnings
from string import ascii_letters, digits, punctuation

from bs4 import BeautifulSoup

warnings.filterwarnings("ignore", category=UserWarning, module='bs4')


class BaseFilter:
    """Base filter class, it does NOTHING.
    This class is design for providing ``__repr__`` string, which is useful in
    python interactive prompt.
    """

    def __repr__(self):
        return '<%s>' % type(self).__name__

    def __call__(self, text: str) -> str:
        return text


class GeneralFilter(BaseFilter):

    def __repr__(self):
        return '<{clsn} repl=`{repl}`>'.format(clsn=type(self).__name__,
                                               repl=self.repl)

    def __call__(self, s: str, count: int = 0):
        return self.pattern.sub(self.repl, s, count)


class EmailFilter(GeneralFilter):
    
    def __init__(self, replace=' '):
        if isinstance(replace, str):
            self.repl = replace
        else:
            self.repl = lambda match: next(replace) # for iterator tag
        self.expression = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
        self.pattern = re.compile(self.expression)


class URLFilter(GeneralFilter):
    
    # copied from string_constants.py
    schemes = list(map(lambda x: '{}:\/\/'.format(x), ("http", "https", "ftp", "sftp", "irc", "magnet", "file", "data"))) + ['www.']
    schemes_lenient = list(map(lambda x: '{}:\/\/'.format(x), ("h?ttp", "h?ttps", "ftp", "sftp", "irc", "magnet", "file", "data"))) + ['www.']

    # from RFC
    gen_delims = ":/?#[]@"
    sub_delims = "!$&'()*+;=%"
    reserved = gen_delims + sub_delims
    unreserved = ascii_letters + digits + "-._~"
    valid_uri_characters = reserved + unreserved

    def __init__(self, replace=' ', allow_common_typoes=True):
        
        if isinstance(replace, str):
            self.repl = replace
        else:
            self.repl = lambda match: next(replace) # for iterator tag

        if allow_common_typoes:
            self.expression = r"((?:{})[{}]+)".format("|".join(self.schemes), re.escape(self.valid_uri_characters))
        else:
            self.expression = r"((?:{})[{}]+)".format("|".join(self.schemes_lenient), re.escape(self.valid_uri_characters))
        self.pattern = re.compile(self.expression)


class EscapeCharFilter(GeneralFilter):

    def __init__(self, replace:str=' '):
        self.repl = replace
        self.expression = r'([{}])'.format('\r\n\t\v')
        self.pattern = re.compile(self.expression)


class InvisableCharFilter(GeneralFilter):
    
    def __init__(self, replace:str='', ):
        self.repl = replace
        self.expression = r'([{}])'.format(r'\xa0\xad\u200c')
        self.pattern = re.compile(self.expression)


class KeyWordsFilter(GeneralFilter):
    
    def __init__(self, replace:str=' ', keywords=('mailto',)):
        self.repl = replace
        self.expression = r'({})'.format('|'.join(map(lambda x: '({})'.format(x), keywords)))
        self.pattern = re.compile(self.expression)


class MultiSpacesFilter(GeneralFilter):
    
    def __init__(self, replace:str=' '):
        self.repl = replace
        self.expression = r'(\s+)'
        self.pattern = re.compile(self.expression)


class FilterNonChars(GeneralFilter):

    def __init__(self, replace:str=' '):
        self.repl = replace
        self.expression = r'(\W+)'
        self.pattern = re.compile(self.expression)
        
        
class HTMLFilter(GeneralFilter):
    def __init__(self, replace:str=''):
        self.repl = replace
        self.expression = "<[^>]*>"
        self.pattern = re.compile(self.expression)

class HTML2Text:
    
    def __init__(self, filters=('script', 'style', 'meta', 'noscript')):
        self.filters = filters

    def extract_text_from_html(self, html:str):

        soup = BeautifulSoup(html, features='lxml')
        for script in soup(self.filters): # remove all javascript and stylesheet code
            script.decompose()
        return soup.get_text()
    
    def __call__(self, html:str):
        try:
            return self.extract_text_from_html(html)
        except:
            return str()



class UnicodeStandarizer:
    
    def __init__(self, method='NFKC'): # NFD, NFC, NFKD, NFKC
        self.method = method
    
    def __call__(self, text:str):
        return unicodedata.normalize(self.method, text)


class Stripper:
    @staticmethod    
    def __call__(text:str):
        return text.strip()


class Lowercasting:
    @staticmethod    
    def __call__(text:str):
        return text.lower()


class SequentialProcessor(BaseFilter):
    """Sequential processor: A class provide an interface to create integral
    processor that would sequentially filter text by the given filters.
    Usage::
        text_preprocessor = SequentialProcessor(
            UnicodeStandarizer(),
            HTML2Text(),
            InvisableCharFilter(),
            LIDFilter(),
            Lowercasting(),
            KeyWordsFilter(),
            MultiSpacesFilter(),
            Stripper(),
        )
    """

    def __init__(self, *processors):
        self.pipeline = processors

    def __repr__(self):
        if len(self.pipeline) == 1:
            children_desc = repr(self.pipeline)
        else:
            # use textwrap to support nested processors
            children_desc = '\n' + ',\n'.join(
                textwrap.indent(repr(p), ' ' * 4)
                for p in self.pipeline)
        return '<{clsn} [{child}]>'.format(clsn=type(self).__name__,
                                           child=children_desc)

    def __call__(self, text: str):
        for processor in self.pipeline:
            text = processor(text)
        return text

class GeneralProcessor(SequentialProcessor):
    pass

class KeepWordSpace(GeneralFilter):
    def __init__(self, replace:str=' '):
        self.repl = replace
        self.expression = r'([^\w\s])'
        self.pattern = re.compile(self.expression)
        
class WordModelPreprocessor(GeneralProcessor):
    
    def __init__(self):
        self.pipeline = (UnicodeStandarizer(), HTML2Text(), URLFilter(), EscapeCharFilter(), KeepWordSpace(), Lowercasting(), MultiSpacesFilter(), Stripper())

class LIDFilter(GeneralFilter):
    
    def __init__(self, replace=' '):
        self.repl = replace
        filters = EmailFilter(), URLFilter(), EscapeCharFilter(), FilterNonChars()
        self.expression = '|'.join(list(map(lambda x: x.expression, filters)))
        self.pattern = re.compile(self.expression)

class TextPreprocessor(GeneralProcessor):
    
    def __init__(self):
        self.pipeline = (UnicodeStandarizer(), HTML2Text(), InvisableCharFilter(), LIDFilter(), Lowercasting(), KeyWordsFilter(), MultiSpacesFilter(), Stripper())

In [10]:
class Segmanent:
    @staticmethod    
    def __call__(text:str):
        text = C.sentence_segmanent(text, return_details=False)
        return text
class SentenceCheck:
    @staticmethod    
    def __call__(text:str):
        text = C.sentence_check(text, return_details=False)[0]
        return text

class TextPreprocessor(GeneralProcessor):
    
    def __init__(self):
        self.pipeline = (UnicodeStandarizer(), HTML2Text(), URLFilter(), EmailFilter(), InvisableCharFilter(), LIDFilter(), Lowercasting(), KeyWordsFilter(), MultiSpacesFilter(), Stripper(), Segmanent(), SentenceCheck())
        
class SimpleTextPreprocessor(GeneralProcessor):
    
    def __init__(self):
        self.pipeline = (UnicodeStandarizer(), HTML2Text(), URLFilter(), EmailFilter(), InvisableCharFilter(), LIDFilter(), Lowercasting(), KeyWordsFilter(), MultiSpacesFilter(), Stripper(), SentenceCheck())

In [11]:
text_preprocessor = TextPreprocessor()
simple_text_preprocessor = SimpleTextPreprocessor()

In [ ]:
from multiprocess import Pool

text_preprocessor = TextPreprocessor()
with Pool() as pool:
    train[COMMENT] = pool.map(simple_text_preprocessor, train[COMMENT])
    test[COMMENT] = pool.map(simple_text_preprocessor, test[COMMENT])

In [ ]:
x_train = train["comment_text"].values
x_test = test["comment_text"].values

In [ ]:
MAX_TOKENS = 30000
MAX_SENTENSE_LEN = 100
EMBEDDING_SIZE_1 = all_embs_1.shape[1]
EMBEDDING_SIZE_2 = all_embs_2.shape[1]
EMBEDDING_SIZE_3 = all_embs_3.shape[1]
EMBEDDING_SIZE = EMBEDDING_SIZE_1 + EMBEDDING_SIZE_2 + EMBEDDING_SIZE_3

In [ ]:
x_train = train["comment_text"].values
y_train = train[LABELS].values
x_test = test["comment_text"].values

tokenizer = text.Tokenizer(num_words=MAX_TOKENS)
tokenizer.fit_on_texts(list(x_train) + list(x_test)) # is it a proper trick?

x_train, x_test = map(tokenizer.texts_to_sequences, [x_train, x_test])
x_train, x_test = map(lambda x: sequence.pad_sequences(x, maxlen=MAX_SENTENSE_LEN), [x_train, x_test])


In [ ]:
word_index = tokenizer.word_index
nb_words = min(MAX_TOKENS, len(word_index))
embedding_matrix_1 = np.random.normal(emb_mean_1, emb_std_1, (nb_words, EMBEDDING_SIZE_1))
embedding_matrix_2 = np.random.normal(emb_mean_2, emb_std_2, (nb_words, EMBEDDING_SIZE_2))
embedding_matrix_3 = np.random.normal(emb_mean_3, emb_std_3, (nb_words, EMBEDDING_SIZE_3))
for word, i in word_index.items():
    if i >= MAX_TOKENS: continue

    embedding_vector_1 = embeddings_index_1.get(word)
    if embedding_vector_1 is not None: 
        embedding_matrix_1[i] = embedding_vector_1
        
    embedding_vector_2 = embeddings_index_2.get(word)
    if embedding_vector_2 is not None: 
        embedding_matrix_2[i] = embedding_vector_2

    embedding_vector_3 = embeddings_index_3.get(word)
    if embedding_vector_3 is not None: 
        embedding_matrix_3[i] = embedding_vector_3

In [ ]:
embedding_matrix_1.shape

In [ ]:
embedding_matrix_2.shape

In [ ]:
embedding_matrix_3.shape

In [23]:
embedding_matrix = np.concatenate((embedding_matrix_1, embedding_matrix_2, embedding_matrix_3), axis=1)
embedding_matrix.shape

(30000, 900)

# Data understanding & visualization

# De-noise (no drop data)

# Feature engineering

# Offline augmentation

# Standarization

# Scaling

# Normalization

# Feature selection

# Data selection

# Optimization

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [28]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=False)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [38]:
MAX_LEN = 100

In [29]:
train_input = bert_encode(train[COMMENT].values, tokenizer, max_len=MAX_LEN)
test_input = bert_encode(test[COMMENT].values, tokenizer, max_len=MAX_LEN)
train_output = train[LABELS].values

In [30]:
import tokenization
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=False)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
# def build_model(bert_layer, MAX_LEN=512):
#     input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
#     input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_mask")
#     segment_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="segment_ids")

#     _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
#     clf_output = sequence_output[:, 0, :]
#     out = Dense(len(LABELS), activation='sigmoid')(clf_output)
    
#     model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
#     model.compile(Adam(lr=3e-4), loss='binary_crossentropy', metrics=['acc'])

#     return model

In [ ]:
# model = build_model(bert_layer, MAX_LEN=100)
# model.summary()

# checkpoint = ModelCheckpoint('bert_model.h5', monitor='val_loss', save_best_only=True)
# earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=1,  verbose=1)
# hist = model.fit(
#     train_input, 
#     train_output,
#     validation_split=0.05,
#     epochs=100,
#     shuffle=True,
#     callbacks=[checkpoint, earlystop],
#     batch_size=256,
#     verbose=1
# )

In [ ]:

input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_mask")
segment_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="segment_ids")
_, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
clf_output = sequence_output[:, 0, :]

x_bert = keras.layers.Dense(MAX_LEN, activation='relu')(clf_output)
x_bert = keras.layers.Dropout(0.2)(x_bert)
x_bert = keras.layers.Dense(MAX_LEN, activation='relu')(x_bert)



model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, mask_zero=True, weights=[embedding_matrix], embeddings_regularizer=l2(1e-5))(model_input)
x = keras.layers.GRU(512, return_sequences=True, activation="relu", kernel_regularizer=l2(1e-5))(x)
x = keras.layers.SpatialDropout1D(0.2)(x)
x = keras.layers.GRU(512, return_sequences=False, activation="relu", kernel_regularizer=l2(1e-5))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(512, activation="relu")(x)


x = tf.keras.layers.concatenate([x, x_bert], axis=-1)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, activation="relu")(x)


model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)


model = Model(inputs=[model_input, input_word_ids, input_mask, segment_ids], outputs=model_output)

# model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])


checkpoint = ModelCheckpoint('bert_model.h5', monitor='val_loss', save_best_only=True)
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=1,  verbose=1)

hist = model.fit([x_train]+list(train_input), 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[checkpoint, earlystop], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('161.submission.csv', index=False)

# ?

Epoch 1/100


/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


248/593 [===========>..................] - ETA: 11:20:59 - loss: 13.1617 - acc: 0.6289

In [35]:
y_train.shape

(159571, 6)

In [36]:
train_output.shape

(159571, 6)

In [ ]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, mask_zero=True, weights=[embedding_matrix], embeddings_regularizer=l2(1e-5))(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('162.submission.csv', index=False)

# ?

In [ ]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.1)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('163.submission.csv', index=False)

# ?

In [ ]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.1)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=32,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('164.submission.csv', index=False)

# ?

In [ ]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN,))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(50, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(6, activation="sigmoid")(x)
model = keras.Model(inputs=model_input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('165.submission.csv', index=False)

# ?

In [ ]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN,))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(50, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(6, activation="sigmoid")(x)
model = keras.Model(inputs=model_input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=32,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('166.submission.csv', index=False)
# ?

# Parameter tuning

# Online augmentation

# Model selection / blending

# Post-processing

# Evaluation

# Reasoning

# Monitoring